In [1]:
import pandas as pd
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/irenetesta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/irenetesta/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
categories = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'hope']
categories_dfs = {}
for category in categories:
	categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)

/var/folders/24/k023d_r17kq_q1b7bj27y7t80000gn/T/ipykernel_1164/445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
/var/folders/24/k023d_r17kq_q1b7bj27y7t80000gn/T/ipykernel_1164/445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
/var/folders/24/k023d_r17kq_q1b7bj27y7t80000gn/T/ipykernel_1164/445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with del

In [4]:
lexicon = pd.DataFrame(columns=['word'])
for category in categories:
    lexicon = pd.merge(lexicon, categories_dfs[category], on='word', how='outer')
lexicon.dropna(inplace=True) # row with empty string
lexicon.sort_values(by='word', inplace=True)
lexicon

,word,anger,disgust,fear,joy,sadness,surprise,hope
11384,aback,0,0,0,0,0,0,0
11830,abacus,0,0,0,0,0,0,0
1324,abandon,0,0,1,0,1,0,0
663,abandoned,1,0,1,0,1,0,0
609,abandonment,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...
5281,zone,0,0,0,0,0,0,0
5458,zoo,0,0,0,0,0,0,0
7907,zoological,0,0,0,0,0,0,0
13967,zoology,0,0,0,0,0,0,0


In [5]:
TRAIN_DATA_PATH = "./datasets/WASSA23_essay_level_with_labels_train.tsv"
train_df = pd.read_csv(TRAIN_DATA_PATH, sep='\t')

In [11]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

preproc_lexicon = {}
lexicon['preproc_word'] = ''
for index, row in lexicon.iterrows():
    lemma = lemmatizer.lemmatize(row['word'])
    stemma = stemmer.stem(lemma)
    lexicon.loc[index, 'preproc_word'] = stemma
    if stemma in preproc_lexicon:
        preproc_lexicon[stemma] += lexicon.loc[index, categories] # |=
    else:
        preproc_lexicon[stemma] = lexicon.loc[index, categories]
preproc_lexicon = pd.DataFrame(preproc_lexicon).T
preproc_lexicon[preproc_lexicon > 0] = 1
preproc_lexicon = preproc_lexicon.loc[(preproc_lexicon!=0).any(axis=1)]
preproc_lexicon.to_csv('./lexicon/preproc_lexicon.csv')
lexicon.to_csv('./lexicon/lexicon.csv', index=False)
preproc_lexicon

,anger,disgust,fear,joy,sadness,surprise,hope
abandon,1,0,1,0,1,1,0
abduct,0,0,1,0,1,1,0
aberr,0,1,0,0,0,0,0
abhor,1,1,1,0,0,0,0
abhorr,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...
youth,1,0,1,1,0,1,1
zani,0,0,0,0,0,1,0
zeal,0,0,0,1,0,1,1
zealou,0,0,0,1,0,0,0


In [10]:
for category in categories:
	train_df[f'{category}_count'] = [0 for _ in range(len(train_df))]

for index, row in train_df.iterrows():
	lemmas = [lemmatizer.lemmatize(word) for word in row['essay'].split()]
	stemmas = [stemmer.stem(lemma) for lemma in lemmas]
	print(stemmas)
	for stemma in stemmas:
		if stemma in preproc_lexicon.index:
			print(stemma)
			for category in categories:
				train_df.loc[index, f'{category}_count'] += preproc_lexicon.loc[stemma][category]
	break
train_df


['it', 'break', 'my', 'heart', 'to', 'see', 'peopl', 'live', 'in', 'those', 'conditions.', 'i', 'hope', 'that', 'all', 'the', 'aid', 'that', 'wa', 'sent', 'to', 'the', 'island', 'make', 'it', 'to', 'the', 'peopl', 'who', 'need', 'it', 'the', 'most.', 'i', 'do', 'not', 'know', 'what', 'i', 'would', 'do', 'it', 'that', 'wa', 'my', 'famili', 'and', 'i.', 'i', 'would', 'hope', 'that', 'i', 'would', 'do', 'my', 'best,', 'but', 'i', 'can', 'see', 'how', 'depress', 'and', 'hopeless', 'you', 'could', 'feel', 'have', 'your', 'whole', 'life', 'chang', 'becaus', 'of', 'a', 'storm', 'and', 'not', 'know', 'where', 'your', 'next', 'meal', 'is', 'come', 'from.']
break
hope
hope
depress
hopeless
feel
chang
storm


,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,split,essay_id,emotion,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count
0,2,35,It breaks my heart to see people living in tho...,6.833333,6.625,30,1,6,3,37,...,train,1,Hope/Sadness,4,3,4,3,3,4,3
1,3,35,I wonder why there aren't more people trying t...,5.833333,6.000,19,1,6,2,32,...,train,2,Anger,0,0,0,0,0,0,0
2,5,35,"After reading the article, you can't help but ...",1.000000,1.375,17,1,6,1,29,...,train,4,Sadness,0,0,0,0,0,0,0
3,6,213,It is so sad that someone who had such an amaz...,6.166667,6.625,16,2,5,1,28,...,train,5,Sadness,0,0,0,0,0,0,0
4,8,213,"From reading the article, it looks like the wo...",6.833333,1.000,30,1,6,3,37,...,train,7,Neutral,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,495,218,I feel that this will become a national proble...,6.500000,6.750,30,1,6,3,37,...,train,994,Neutral,0,0,0,0,0,0,0
788,496,103,The whole situation is sketchy. The wavering r...,3.166667,6.375,16,2,5,1,28,...,train,995,Neutral,0,0,0,0,0,0,0
789,498,103,The death of a former aide to Russian Presiden...,6.000000,2.000,43,2,6,1,33,...,train,997,Neutral,0,0,0,0,0,0,0
790,499,103,Everything about Russia really freaks me out. ...,6.000000,6.000,53,2,3,1,27,...,train,998,Fear,0,0,0,0,0,0,0


In [19]:
from textblob import TextBlob
print(TextBlob('think').sentiment.subjectivity)
print(TextBlob('I think so').sentiment.subjectivity)
print(TextBlob('I think so!').sentiment.subjectivity)
print(TextBlob('Textblob is amazingly simple to use. What great fun!').sentiment.subjectivity)
count = 0
words = 'Textblob is amazingly simple to use. What great fun!'.split()
for word in words:
    count += TextBlob(word).sentiment.subjectivity
count /= len(words)
print(count)

0.0
0.0
0.0
0.4357142857142857
0.24523809523809526
